In [1]:
import time
import numpy as np
from tqdm.notebook import tqdm

from components.dataset_jo import *

dataset = Dataset_subjectDependent('data')
segment_lenght = 4 #second
dataset.set_segment(7680//(128*segment_lenght))
for filename in dataset.get_file_list():
    data, labels, _ = dataset.get_data(filename, return_type='numpy')
    print(filename, labels.shape, labels.sum(axis=0)/labels.shape[0]*100)
    break

Found: 32 files
s01 (600, 2) [47.5 60. ]


In [2]:
def train_model(X_ori,y_ori,groups_ori,filename=None, kernel='rbf',return_text=False):
    # Make a copy because I am paranoid
    X,y,groups = X_ori.copy(), y_ori.copy(), groups_ori.copy()

    from sklearn.svm import SVC
    from sklearn.model_selection import GroupShuffleSplit 
    from sklearn.model_selection import cross_val_score

    model = SVC(kernel=kernel,max_iter=50000)
    cv = GroupShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
    cross = cross_val_score(model, X, y, cv=cv,groups=groups, n_jobs=8)
    
    # We probably dont need this
    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X, y)
    ans = model.predict(X)
    acc = sum(ans == y) / len(y)
    # If the model answer with all 0 or 1, we print this message
    text = None
    if( sum(ans) == len(y) or sum(ans) == 0 ): 
        text = f"-----WARNING: Model {filename} failed to learn: sum(ans)={sum(ans)} sum(y)={sum(y)} len(y)={len(y)}"
    if(return_text):
        return model, acc, cross, text
    else:
        if(text != None): print(text)
        return model, acc, cross

## 2. Spectral Features

In [3]:
from mne_features.feature_extraction import FeatureExtractor

In [4]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})


accs,cv_means,cv_stds  = [],[],[]
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

accs_v,cv_means_v,cv_stds_v  = [],[],[]
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.705|10-CV=0.536|STD=0.14171|Time spend=32.432703495025635
	AROUSAL-s02|Acc=0.64833|10-CV=0.544|STD=0.08079|Time spend=0.6004517078399658
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=120.0 len(y)=600
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.5529723167419434
	AROUSAL-s04|Acc=0.665|10-CV=0.53867|STD=0.10254|Time spend=0.5832347869873047
	AROUSAL-s05|Acc=0.585|10-CV=0.4|STD=0.09428|Time spend=0.5917432308197021
	AROUSAL-s06|Acc=0.62167|10-CV=0.524|STD=0.13863|Time spend=0.570556640625
	AROUSAL-s07|Acc=0.685|10-CV=0.634|STD=0.10496|Time spend=0.5847246646881104
	AROUSAL-s08|Acc=0.69667|10-CV=0.42533|STD=0.1188|Time spend=0.5811765193939209
	AROUSAL-s09|Acc=0.65333|10-CV=0.49467|STD=0.08038|Time spend=0.5979676246643066
	AROUSAL-s10|Acc=0.69667|10-CV=0.55333|STD=0.09494|Time spend=0.6734280586242676
	AROUSAL-s11|Acc=0.63|10-CV=0.59533|STD=0.05918|Time spend=0.6626689434051514
-----WARNING: Model s12 failed to learn: sum(ans)=600.0 sum(y)=495

## 3. Asymetry

In [5]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")

left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


In [6]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tAROUSAL-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs[feature_name].append(acc)
        cv_means[feature_name].append(cross.mean())
        cv_stds[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"AROUSAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

accs_v,cv_means_v,cv_stds_v  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tVALENCE-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs_v[feature_name].append(acc)
        cv_means_v[feature_name].append(cross.mean())
        cv_stds_v[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"VALENCE-{feature_name}|Acc={sum(accs_v[feature_name])/len(accs_v[feature_name])}|10-CV={sum(cv_means_v[feature_name])/len(cv_means_v[feature_name])}|STD={sum(cv_stds_v[feature_name])/len(cv_stds_v[feature_name])}")

for feature_name in ['DASM','RASM','DCAU']:
    accs[feature_name].extend(accs_v[feature_name])
    cv_means[feature_name].extend(cv_means_v[feature_name])
    cv_stds[feature_name].extend(cv_stds_v[feature_name])
    print(f"TOTAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

-----WARNING: Model s01 failed to learn: sum(ans)=600.0 sum(y)=360.0 len(y)=600
	AROUSAL-s01-DASM|Acc=0.6|10-CV=0.538|STD=0.13206|Time spend=0.36437177658081055
	AROUSAL-s01-RASM|Acc=0.61333|10-CV=0.558|STD=0.15409|Time spend=0.4083542823791504
	AROUSAL-s01-DCAU|Acc=0.60167|10-CV=0.51667|STD=0.11878|Time spend=0.4462447166442871
	AROUSAL-s02-DASM|Acc=0.63|10-CV=0.53933|STD=0.08986|Time spend=0.3340623378753662
	AROUSAL-s02-RASM|Acc=0.60333|10-CV=0.49867|STD=0.16825|Time spend=0.37640976905822754
	AROUSAL-s02-DCAU|Acc=0.63167|10-CV=0.53733|STD=0.08714|Time spend=0.41666555404663086
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=120.0 len(y)=600
	AROUSAL-s03-DASM|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.34456801414489746
	AROUSAL-s03-RASM|Acc=0.80667|10-CV=0.72933|STD=0.15538|Time spend=0.37246155738830566
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=120.0 len(y)=600
	AROUSAL-s03-DCAU|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.4097278118133545
	AROUSAL-s

## 4. Connectivity

### 4.1 $ \text{PCC}_{\text{time}}(i,j) = \frac{\text{Cov}[\mathbf{X}_i, \mathbf{X}_j]}{\sigma_{\mathbf{X}_i} \sigma_{\mathbf{X}_j}} $

In [7]:

def pearson_correlation(x,y):
    """ x,y denoted the signal_x and signal_y following the equation """
    cov = np.cov(x, y)
    # print(cov)
    # [[ 8806859.74527069  8007149.0906219 ] ==> [[cov_xx, cov_xy]
    # [ 8007149.0906219  10396797.72458848]]      [cov_yx, cov_yy]]
    cov_xy = cov[0,1] # or cov[1,0]
    cov_xx = cov[0,0]
    cov_yy = cov[1,1]
    corr = cov_xy / ( cov_xx**0.5 * cov_yy**0.5  )
    return corr

def _cal(p_id, partial_data):
    # print(f"p_id:{p_id} - data to run {partial_data.shape}")
    from itertools import combinations
    pcc = []
    for index in range(partial_data.shape[0]):
        pcc_epoch = []
        for comb in combinations(list(range(partial_data.shape[1])), 2):
            pcc_ab = pearson_correlation(partial_data[index, comb[0], :], partial_data[index, comb[1], :]   )
            pcc_epoch.append(pcc_ab)
        pcc_epoch = np.hstack(pcc_epoch)
        pcc.append(pcc_epoch)
    pcc = np.vstack(pcc)
    return pcc

def calculate_pcc(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(_cal, [p_id, data[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    except Exception as e:
        print(e)
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)

In [8]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    data_csd = mne.preprocessing.compute_current_source_density(data)
    pcc = calculate_pcc(data_csd.get_data())
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.69833|10-CV=0.53533|STD=0.13708|Time spend=3.038990020751953
	VALENCE-s01|Acc=0.64167|10-CV=0.452|STD=0.08798|Time spend=3.4338572025299072
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.60833|10-CV=0.55467|STD=0.10958|Time spend=2.962812662124634
	VALENCE-s02|Acc=0.59167|10-CV=0.44|STD=0.08584|Time spend=3.3282783031463623
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=2.814911127090454
	VALENCE-s03|Acc=0.55|10-CV=0.41933|STD=0.11433|Time spend=3.1861572265625
Fitted sphere radius:         95.3 mm
Origin head coordina

In [9]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.69833|10-CV=0.53533|STD=0.13708|Time spend=3.038990020751953
	VALENCE-s01|Acc=0.64167|10-CV=0.452|STD=0.08798|Time spend=3.4338572025299072
	AROUSAL-s02|Acc=0.60833|10-CV=0.55467|STD=0.10958|Time spend=2.962812662124634
	VALENCE-s02|Acc=0.59167|10-CV=0.44|STD=0.08584|Time spend=3.3282783031463623
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=2.814911127090454
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=120.0 len(y)=600
	VALENCE-s03|Acc=0.55|10-CV=0.41933|STD=0.11433|Time spend=3.1861572265625
-----WARNING: Model s03 failed to learn: sum(ans)=600.0 sum(y)=330.0 len(y)=600
	AROUSAL-s04|Acc=0.6|10-CV=0.54|STD=0.22|Time spend=2.887798547744751
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=240.0 len(y)=600
	VALENCE-s04|Acc=0.6|10-CV=0.49|STD=0.18682|Time spend=3.2484521865844727
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=240.0 len(y)=600
	AROUSAL-s05|Acc=0.57333|10-CV=0.418|STD=0.10058|Time spend=2.933106422424316

### 4.2 $ \text{PCC}_{freq} (i,j) = \frac{\text{Cov}[\hat{\mathbf{X}}_i, \hat{\mathbf{X}}_j]}{\sigma_{\hat{\mathbf{X}}_i} \sigma_{\hat{\mathbf{X}}_j}} $

In [10]:
def calculate_fft(signal, sfreq):
    """ signal: can be 1D array of (n_sample,) or 2D array of (n_signal, n_sample)  """
    number_sample = signal.shape[-1]

    # the result will be a complex number. We can obtain the magnitude using `absolute`
    magnitude = np.abs(np.fft.fft(signal))
    # scale the result
    magnitude = magnitude / (number_sample/2)
    # Selecting the range
    magnitude = magnitude.T[:number_sample//2].T
    freq_range = np.fft.fftfreq(number_sample, d=1/sfreq)[:number_sample//2]

    return magnitude, freq_range

In [11]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)
    magnitude, freq_range = calculate_fft(data_csd.get_data(), 128)
    pcc = calculate_pcc(magnitude)
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.79|10-CV=0.53533|STD=0.12289|Time spend=3.028975009918213
	VALENCE-s01|Acc=0.77333|10-CV=0.48867|STD=0.07118|Time spend=3.429551362991333
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.64167|10-CV=0.574|STD=0.10424|Time spend=2.9688048362731934
	VALENCE-s02|Acc=0.62|10-CV=0.45267|STD=0.08514|Time spend=3.3559300899505615
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=3.2946813106536865
	VALENCE-s03|Acc=0.63333|10-CV=0.456|STD=0.13291|Time spend=3.741478681564331
Fitted sphere radius:         95.3 mm
Origin head coordin

In [12]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.79|10-CV=0.53533|STD=0.12289|Time spend=3.028975009918213
	VALENCE-s01|Acc=0.77333|10-CV=0.48867|STD=0.07118|Time spend=3.429551362991333
	AROUSAL-s02|Acc=0.64167|10-CV=0.574|STD=0.10424|Time spend=2.9688048362731934
	VALENCE-s02|Acc=0.62|10-CV=0.45267|STD=0.08514|Time spend=3.3559300899505615
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=3.2946813106536865
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=120.0 len(y)=600
	VALENCE-s03|Acc=0.63333|10-CV=0.456|STD=0.13291|Time spend=3.741478681564331
	AROUSAL-s04|Acc=0.63333|10-CV=0.49667|STD=0.1343|Time spend=3.362854480743408
	VALENCE-s04|Acc=0.61|10-CV=0.48733|STD=0.11751|Time spend=3.77034854888916
	AROUSAL-s05|Acc=0.65667|10-CV=0.45067|STD=0.11501|Time spend=3.031146287918091
	VALENCE-s05|Acc=0.62833|10-CV=0.55333|STD=0.07333|Time spend=3.414764404296875
	AROUSAL-s06|Acc=0.68667|10-CV=0.55133|STD=0.14236|Time spend=3.171693801879883
	VALENCE-s06|Acc=0.75|10-CV=0.71733|STD=0.0731|Time spend=3.5

### 4.3 $ \text{PLV}(j,k) = \frac{1}{T} | \Sigma^{T}_{t=1} e^{i(\phi^{t}_{j} - \phi^{t}_{k})}   | $

In [13]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLV_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    print(phase.shape)
    from itertools import combinations
    plv = []
    # count = 0
    for index in range(phase.shape[0]):
        plv_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            plv_ab = np.abs(np.average(np.exp(complex(0,1) * phase_diff), axis=1))
            plv_epoch.append(plv_ab)
        plv_epoch = np.vstack(plv_epoch)
        # print(plv_epoch.shape) => (300, 32, 65, 23)
        plv_epoch_5 = np.concatenate([ plv_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        plv.append(np.expand_dims(plv_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    plv = np.vstack( plv )
    return plv.squeeze()

def calculate_plv(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLV_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [14]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    plv = calculate_plv(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(plv, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(plv, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
========= close ========
	AROUSAL-s01|Acc=0.83333|10-CV=0.56333|STD=0.1446|Time spend=21.29649829864502
	VALENCE-s01|Acc=0.83167|10-CV=0.48533|STD=0.07338|Time spend=23.56507420539856
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
(75, 32, 65, 87)
========= close ========
	AROUSAL-s02|Acc=0.70333|10-CV=0.54467|STD=0.09367|Time spend=22.981752634048462
	VALENCE-s02|Acc=0.74833|10-CV=0.67267|STD=0.10597|Time spend=24.829442977905273
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Ori

In [15]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.83333|10-CV=0.56333|STD=0.1446|Time spend=21.29649829864502
	VALENCE-s01|Acc=0.83167|10-CV=0.48533|STD=0.07338|Time spend=23.56507420539856
	AROUSAL-s02|Acc=0.70333|10-CV=0.54467|STD=0.09367|Time spend=22.981752634048462
	VALENCE-s02|Acc=0.74833|10-CV=0.67267|STD=0.10597|Time spend=24.829442977905273
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=22.921663522720337
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=120.0 len(y)=600
	VALENCE-s03|Acc=0.74667|10-CV=0.59333|STD=0.18304|Time spend=24.9835102558136
	AROUSAL-s04|Acc=0.735|10-CV=0.56|STD=0.10758|Time spend=22.24033546447754
	VALENCE-s04|Acc=0.74167|10-CV=0.608|STD=0.08331|Time spend=24.269334316253662
	AROUSAL-s05|Acc=0.70167|10-CV=0.44533|STD=0.10383|Time spend=21.771145820617676
	VALENCE-s05|Acc=0.675|10-CV=0.548|STD=0.08863|Time spend=23.92218852043152
	AROUSAL-s06|Acc=0.635|10-CV=0.51133|STD=0.12817|Time spend=21.1551251411438
	VALENCE-s06|Acc=0.75|10-CV=0.72|STD=0.07483|Time spend=22.9

### 4.4 $ \text{PLI}(j,k) =  \frac{1}{T} | \Sigma^{T}_{t=1} \text{sign}(Im[e^{i (\phi^{t}_{j} - \phi^{t}_{k})}])  | $

In [16]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLI_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    from itertools import combinations
    pli = []
    # count = 0
    for index in range(phase.shape[0]):
        pli_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            pli_ab = np.abs(np.average(   np.sign(np.imag(   np.exp(complex(0,1) * phase_diff) ))  , axis=1))
            pli_epoch.append(pli_ab)
        pli_epoch = np.vstack(pli_epoch)
        pli_epoch_5 = np.concatenate([ pli_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        pli.append(np.expand_dims(pli_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    pli = np.vstack( pli )
    return pli.squeeze()

def calculate_pli(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLI_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [17]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    pli = calculate_pli(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(pli, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pli, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.955|10-CV=0.53133|STD=0.12589|Time spend=21.37575078010559
	VALENCE-s01|Acc=1.0|10-CV=0.43733|STD=0.05555|Time spend=23.73000979423523
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.86667|10-CV=0.54333|STD=0.1009|Time spend=24.265512943267822
	VALENCE-s02|Acc=0.9|10-CV=0.47733|STD=0.10125|Time spend=26.578510999679565
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.84833|10-CV=0.73|STD=0.15524|Time spend=22.027878046035767
	VALENCE-s03|Acc=0.98167|10-CV=0.434|STD=0.11927|Time spend=24.609827041625977
Fitted sphere radius:         95.3 mm
Origin head coord

In [18]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.955|10-CV=0.53133|STD=0.12589|Time spend=21.37575078010559
	VALENCE-s01|Acc=1.0|10-CV=0.43733|STD=0.05555|Time spend=23.73000979423523
	AROUSAL-s02|Acc=0.86667|10-CV=0.54333|STD=0.1009|Time spend=24.265512943267822
	VALENCE-s02|Acc=0.9|10-CV=0.47733|STD=0.10125|Time spend=26.578510999679565
	AROUSAL-s03|Acc=0.84833|10-CV=0.73|STD=0.15524|Time spend=22.027878046035767
	VALENCE-s03|Acc=0.98167|10-CV=0.434|STD=0.11927|Time spend=24.609827041625977
	AROUSAL-s04|Acc=0.86833|10-CV=0.534|STD=0.13385|Time spend=20.901691913604736
	VALENCE-s04|Acc=0.84167|10-CV=0.512|STD=0.12511|Time spend=23.19629406929016
	AROUSAL-s05|Acc=0.89333|10-CV=0.43067|STD=0.10825|Time spend=20.63515067100525
	VALENCE-s05|Acc=0.75167|10-CV=0.584|STD=0.088|Time spend=22.94931125640869
	AROUSAL-s06|Acc=0.955|10-CV=0.49667|STD=0.11475|Time spend=21.73943543434143
	VALENCE-s06|Acc=0.86333|10-CV=0.72|STD=0.07483|Time spend=24.056525230407715
	AROUSAL-s07|Acc=0.96667|10-CV=0.63067|STD=0.11874|Time spend=2

## 5. CSP

In [19]:
from components.CSP import CSP
from mne_features.feature_extraction import FeatureExtractor

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

In [20]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape[1] == 160
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s01|Acc=0.795|10-CV=0.75133|STD=0.06491|Time spend=1.1227118968963623


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s02|Acc=0.61333|10-CV=0.54067|STD=0.08816|Time spend=3.2156808376312256


...

	AROUSAL-s30|Acc=0.89167|10-CV=0.864|STD=0.05934|Time spend=0.9396429061889648


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s31|Acc=0.59833|10-CV=0.502|STD=0.10688|Time spend=0.6962201595306396


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s32|Acc=0.92167|10-CV=0.92933|STD=0.032|Time spend=0.5935401916503906
AROUSAL|Acc=0.7908854166666667|10-CV=0.7514583333333333|STD=0.07088855053441581


In [21]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape[1] == 160
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s01|Acc=0.8|10-CV=0.772|STD=0.03919|Time spend=0.5979387760162354


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s02|Acc=0.60667|10-CV=0.472|STD=0.09706|Time spend=0.6265716552734375


...

-----WARNING: Model s30 failed to learn: sum(ans)=600.0 sum(y)=405.0 len(y)=600
	VALENCE-s30|Acc=0.675|10-CV=0.61867|STD=0.1463|Time spend=0.6529767513275146


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s31|Acc=0.57667|10-CV=0.474|STD=0.10042|Time spend=0.6511485576629639


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s32|Acc=0.63|10-CV=0.54|STD=0.0996|Time spend=0.6143550872802734
VALENCE|Acc=0.7305208333333334|10-CV=0.6554166666666665|STD=0.08886716269771341


In [22]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.760703125|10-CV=0.7034374999999999|STD=0.07987785661606457
